In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
c_folder="../../data/calibration/"
d_folder="../../data/runData/"
r_n=307
p_folder="../../plots/run_"+str(r_n)+"/"

In [4]:
# helper functions
def choose_bar(df,layer, strip): #works
    df=df[df["strip"]==strip]
    df=df[df["layer"]==layer]
    return df

def import_data(calibration_folder, data_folder, run_number, is_it_pulsed=False): # works for both pulsed and not-pulsed
    
    pedestal_df=pd.read_csv(calibration_folder+"pedestal_fixed.csv", sep=',')
    mip_df=pd.read_csv(calibration_folder+"mip.csv", sep=',')
    
    if is_it_pulsed==True:
        run_df=pd.read_csv(data_folder+"run_"+str(run_number)+"_pulse.csv", sep=',')
        run_df.drop(["pf_spill", "pf_ticks"], axis=1, inplace=True) # we never need these
        run_df.drop(["toa_end0", "toa_end1"], axis=1, inplace=True) # unreliable guys, but we might need them in the future
    else:
        run_df=pd.read_csv(data_folder+"run_"+str(run_number)+".csv", sep=',')
        run_df.drop(["pf_spill", "pf_ticks"], axis=1, inplace=True)
        run_df.drop(["toa_end0", "toa_end1"], axis=1, inplace=True)
        
    return pedestal_df,mip_df,run_df

In [5]:
def drop_extra_pedestal_data(merged_df):
    merged_df.drop(["pedestal_end0", "pedestal_end1",
                    "pedestal_per_time_sample_end0","pedestal_per_time_sample_end1"], axis=1, inplace=True)
    return merged_df

In [6]:
def subtract_pedestals(merged_df, is_it_pulsed=False):
    merged_df["adc_sum_end0"]-=merged_df["pedestal_end0"]
    merged_df["adc_sum_end1"]-=merged_df["pedestal_end1"]
    
    merged_df["adc_max_end0"]-=merged_df["pedestal_per_time_sample_end0"]
    merged_df["adc_max_end1"]-=merged_df["pedestal_per_time_sample_end1"]
    
    merged_df["adc_mean_end0"]-=merged_df["pedestal_per_time_sample_end0"]
    merged_df["adc_mean_end1"]-=merged_df["pedestal_per_time_sample_end1"]

    if is_it_pulsed==True:
        for i in range(8):
            adc_end0_col = f'adc_{i}_end0'
            adc_end1_col = f'adc_{i}_end1'
        
            merged_df[adc_end0_col]-=merged_df["pedestal_per_time_sample_end0"]
            merged_df[adc_end1_col]-=merged_df["pedestal_per_time_sample_end1"]
            
    merged_df=drop_extra_pedestal_data(merged_df)
    
    return merged_df

In [32]:
def select_bars_with_data(run_df, pedestal_df, mip_df=None, subtract_pedestal=False, convert_mip=False, is_it_pulsed=False):
    
    merged_df=run_df.merge(pedestal_df, how='left', on=['layer', 'strip'])
    
    a=merged_df[(merged_df["adc_sum_end0"]>5*merged_df["std_dev_end0"]) & (merged_df["adc_sum_end1"]>5*merged_df["std_dev_end1"])]

    a.drop(["std_dev_end0", "std_dev_end1"], axis=1, inplace=True)

    if subtract_pedestal==True:
        a=subtract_pedestals(a, is_it_pulsed)

    # if convert_mip==True: #similarily
    return a

In [33]:
p,m,r=import_data(c_folder,d_folder, 300, is_it_pulsed=True)

In [35]:
r=select_bars_with_data(r,p, subtract_pedestal=True, is_it_pulsed=True) #this needs to be run as it's own cell

In [36]:
r.sample(10)

,pf_event,tot_end0,adc_sum_end0,adc_mean_end0,adc_max_end0,adc_0_end0,adc_1_end0,adc_2_end0,adc_3_end0,adc_4_end0,adc_5_end0,adc_6_end0,adc_7_end0,layer,strip,tot_end1,adc_sum_end1,adc_mean_end1,adc_max_end1,adc_0_end1,adc_1_end1,adc_2_end1,adc_3_end1,adc_4_end1,adc_5_end1,adc_6_end1,adc_7_end1,pedestal_per_time_sample_std_dev_end0,pedestal_per_time_sample_std_dev_end1
4111087,44790,0,-11,-1.375,12,-4,-22,-12,6,12,7,2,0,8,1,0,3,0.375,14,-3,-20,-10,6,14,10,4,2,2.050624,1.731764
3561899,68820,0,10,1.250,22,-3,-27,-16,2,16,22,10,6,7,1,0,6,0.750,22,-4,-26,-17,2,15,22,8,6,1.890358,2.016687
6360251,4826,0,4,0.500,2,2,0,2,-2,0,0,0,2,11,5,0,1,0.125,2,-1,2,0,1,0,0,0,-1,1.385571,1.870940
158758,17606,0,16,2.000,12,3,-9,-9,0,11,12,4,4,1,2,0,3,0.375,13,1,-12,-10,-1,8,13,2,2,1.730582,1.697010
1122887,57104,0,7,0.875,11,0,-10,-12,4,11,8,2,4,2,7,0,26,3.250,15,0,-10,-14,8,15,12,7,8,1.871623,1.482843
8965937,43102,0,3,0.375,2,0,0,-1,2,2,0,0,0,14,5,0,-5,-0.625,2,-2,-2,-3,-2,0,0,2,2,2.279809,1.568788
8619379,54740,0,-1,-0.125,3,-1,0,0,-1,0,-2,3,0,14,0,0,-4,-0.500,2,-2,-2,-2,-2,0,2,0,2,1.325802,1.668332
11970876,49303,0,6,0.750,3,0,-1,0,0,1,3,1,2,17,11,0,-13,-1.625,0,-2,-1,-2,-2,-2,-2,-2,0,1.411269,1.885008
9859680,4871,0,9,1.125,2,2,1,1,2,2,1,0,0,15,6,0,20,2.500,4,2,2,3,2,4,2,2,3,1.507065,1.449710
9285046,1081,0,14,1.750,3,2,2,2,1,3,1,2,1,14,10,0,-3,-0.375,0,-2,-1,0,0,0,0,0,0,1.485455,1.079136
